In [1]:
# STANDARD LIBRARY IMPORTS
import re
import csv
import os
import datetime
import subprocess
import sys # for args

# THIRD PARTY IMPORTS
import praw

# PROJECT IMPORTS
from setup import reddit
from settings import *
from tools import *
from inbox import check_messages
from reminders import send_reminders
from log_missed_comment import manually_award_comment

subreddit = reddit.subreddit(TARGET_SUBREDDIT)

Loaded 22 users who have opted out from follow-ups.
Loaded 18 users who have opted out from reminders.


Version 7.5.0 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


# Step 1: Get the last awarded post

In [2]:
# get the last awarded post. We don't need to look back past this time.
last_award = next( reddit.user.me().comments.new() )

In [3]:
print( last_award.permalink )

/r/photocritique/comments/1jlc45q/union_college_in_schenectady_ny_critiques/mk7gxob/


# Step 2: Search the most recent 1,000 comments, and record their IDs

In [4]:
num_processed_comments = 0
num_awarded_comments   = 0

# we can only go back 1,000 comments :|
for awarding_comment in subreddit.comments(limit=None):
    num_processed_comments += 1
    
    if (awarding_comment.author is not None and awarding_comment.author.name != 'AutoModerator') and comment_contains_token( awarding_comment ):
        print(f'Should we award points based on comment {awarding_comment.id} by u/{awarding_comment.author.name}?')
        num_awarded_comments += 1

    if awarding_comment.created_utc <= last_award.created_utc:
        # we've gone back far enough
        print('Yay! There were less than 1,000 comments since the bot crashed.\n')
        break
        
print(f'\n\nSearched {num_processed_comments} comments. Found {num_awarded_comments} that might need awarding.')
        
print(f'There is a gap of {( awarding_comment.created_utc - last_award.created_utc ) / 60 / 60:.1f} hours between the last comment we searched, and the last award.')
print('Hopefully the above number is 0 or small!')

Should we award points based on comment mldrp39 by u/linklocked?
Should we award points based on comment mlcmnpz by u/baska_rhymes?
Should we award points based on comment mlceaam by u/baska_rhymes?
Should we award points based on comment mlbp9dc by u/Far-FarmGoose?
Should we award points based on comment mlbj799 by u/No_Split5962?
Should we award points based on comment mlb1m0k by u/itsntcharlie?
Should we award points based on comment mlayjwt by u/RedBoxtops?
Should we award points based on comment mlashlb by u/itsntcharlie?
Should we award points based on comment mlarlrx by u/itsntcharlie?
Should we award points based on comment mlar58s by u/itsntcharlie?
Should we award points based on comment mlaqi9t by u/itsntcharlie?
Should we award points based on comment mlao9zm by u/linklocked?
Should we award points based on comment mla9tg6 by u/MTXD_FTW?
Should we award points based on comment mla9hh4 by u/MTXD_FTW?
Should we award points based on comment mla979q by u/MTXD_FTW?
Should we aw

# Step 3: Manually award all the comments above.

Reddit will rate limit you at some point, so you'll need to wait a few mins if you have more than ~15 comments to award.

To award comments, copy the ID above into the below string, and run the cell. Repeat for all comments listed above. You may want to manually check things like:

- don't award to top level comments? 

Are these things still in `run.py`?

In [5]:
from time import sleep

In [14]:
missing_awards = [
'mkocpjz',
'mkoctyh',
'mkh2nov',
'mkh2kgr',
'mki9f91',
'mkd9kme',
'mkdfn4z',
'mkyppnp',
'mkbs1p6',
'mkg84sa',
'mk7ymdy',
'mkagwbf',
'mkao0vo',
]

In [15]:
for award_id in missing_awards:
    manually_award_comment(award_id)
    sleep(60)

Manually awarding points based on comment mkocpjz by u/CaickPhotography.
Manually awarding points based on comment mkoctyh by u/CaickPhotography.
Manually awarding points based on comment mkh2nov by u/fiyoOnThebayou.
Manually awarding points based on comment mkh2kgr by u/fiyoOnThebayou.
Manually awarding points based on comment mki9f91 by u/fiyoOnThebayou.
Manually awarding points based on comment mkd9kme by u/Either_Surround_7658.
Manually awarding points based on comment mkdfn4z by u/Either_Surround_7658.
Manually awarding points based on comment mkyppnp by u/Caeslius.
Manually awarding points based on comment mkbs1p6 by u/Caeslius.
Manually awarding points based on comment mkg84sa by u/Caeslius.
Manually awarding points based on comment mk7ymdy by u/Particular_View4059.
Manually awarding points based on comment mkagwbf by u/Particular_View4059.
Manually awarding points based on comment mkao0vo by u/Particular_View4059.


# Step 5: Manually look for missed awards that need checking

I would do this by starting at /new, scrolling back to post from the below last-checked comment, and then manually opening all the posts after that and searching for `!CritiquePoint`. Do this until you are fairly certain you've gone far enough.

In [15]:
print('This is the last comment we were able to check:')
print(awarding_comment.permalink)

print('\nComments older than this may need checking manually.')

This is the last comment we were able to check:
/r/photocritique/comments/xza4r1/feedback_request_on_portrait/irmugrd/

Comments older than this may need checking manually.


In [42]:
#  manually_award_comment('irah590')

Manually awarding points based on comment irah590 by u/Koosen-.


In [ ]:
# https://old.reddit.com/r/photocritique/comments/xwqlb5/my_thought_was_better_than_the_execution/irah590/